<a href="https://colab.research.google.com/github/PabloGorSan/AlgortimosDeOptimizacion/blob/main/Copia_de_Seminario_Algoritmos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos: Pablo Gordillo Sánchez   <br>
Url: https://github.com/PabloGorSan/AlgortimosDeOptimizacion/blob/main/Copia_de_Seminario_Algoritmos.ipynb<br>
Problema:

Combinar cifras y operaciones

Descripción del problema:(copiar enunciado)

- El problema consiste en analizar el siguiente problema y diseñar un algoritmo que lo resuelva.
- Disponemos de las 9 cifras del 1 al 9 (excluimos el cero) y de los 4 signos básicos de las
operaciones fundamentales: suma(+), resta(-), multiplicación(*) y división(/)
- Debemos combinarlos alternativamente sin repetir ninguno de ellos para obtener una cantidad
dada. Un ejemplo sería para obtener el 4:
4+2-6/3*1 = 4
- Debe analizarse el problema para encontrar todos los valores enteros posibles planteando las siguientes cuestiones:
 - ¿Qué valor máximo y mínimo se pueden obtener según las condiciones del problema?
 - ¿Es posible encontrar todos los valores enteros posibles entre dicho mínimo y máximo?
                                        

In [79]:
import bisect

(*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br>







Para este problema debemos definir previamente qué significa que no tenga restricciones, ya que podemos entenderlo de distinta manera y obtener una cantidad de posibilidades diversas. Las restricciones que plantea el problema son:
- Debemos combinar números y operaciones alternativamente
- Sin repetir ninguno de ellos.

Empezando por la segunda restricción, si esta no se cumpliera, el espacio de búsqueda para este problema sería infinito, ya que podríamos siempre seguir creando nuevas posibilidades añadiendo una nueva cifra o una nueva operación según corresponda. Por tanto, no vamos a tomarlo tanto como una restricción sino como una característica de nuestro problema (igual que utilizar las cifras y las operaciones básicas).

La primera restricción es más interesante. Anular esta restricción nos sigue limitando el espacio de búsqueda, aunque añade combinaciones no validas (como 2*/3). Teniendo en cuenta las no validas, las posibilidades totales serían igual a 13!, ya que sería una permutación sin repetición de 13 signos (9 dígitos y 4 operaciones). El siguiente código obtiene cuantas de estas combinaciones son validas.


¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones?

Al igual que hemos deducido el número de posibilidades en el apartado anterior, podemos aplicar el mismo razonamiento para obtener el total de posibilidades teniendo en cuenta las restricciones.
En un comienzo debemos elegir una de entre 9 cifras para probar, luego una entre 4 operaciones, una entre 8 cifras, una entre 3 operaciones... hasta quedarnos sin operaciones, por lo que ya solo podremos añadir una cifra más. Por tanto el número de posibilidades es 9 * 4 * 8 * 3 * 7 * 6 * 2 * 5 * 1 = 9! = 362880.

Modelo para el espacio de soluciones<br>
(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)


Respuesta

Según el modelo para el espacio de soluciones<br>
(*)¿Cual es la función objetivo?

(*)¿Es un problema de maximización o minimización?

Respuesta

Diseña un algoritmo para resolver el problema por fuerza bruta

Respuesta

In [80]:
cifras = [str(x) for x in range(1, 10)]
operaciones = ['+', '-', '*', '/']

valores = {
    'posibilidades' : 0,
    'max' : -9999999,
    'min' : 9999999,
    'valores enteros' : [],
    'valores intermedios' : True
}

def fuerzaBrutaRestricciones(cifras, operaciones, cadena, valores):

  calcularValoresEnteros(cifras, operaciones, "", valores)
  return encontrarValoresIntermedios(valores)

def calcularValoresEnteros(cifras, operaciones, cadena, valores):
  for c in cifras:
    if len(operaciones) == 0:
      valor = eval(cadena + c)
      valores['posibilidades'] += 1
      if valor.is_integer():
        if valor not in valores['valores enteros']:
          bisect.insort(valores['valores enteros'], valor)
        if valor > valores['max'] : valores['max'] = valor
        if valor < valores['min'] : valores['min'] = valor
    else:  
      for o in operaciones:
        cifrasMenos1 = cifras.copy()
        cifrasMenos1.remove(c)
        operacionesMenos1 = operaciones.copy()
        operacionesMenos1.remove(o)
        calcularValoresEnteros(cifrasMenos1, operacionesMenos1, cadena + c + o, valores)

def encontrarValoresIntermedios(valores):
    i = 0
    while i < len(valores['valores enteros'])-1 and valores['valores intermedios']:
      if abs(valores['valores enteros'][i] - valores['valores enteros'][i]+1) != 1: valores['valores intermedios'] = False
      else: i += 1

    return valores

valores = fuerzaBrutaRestricciones(cifras, operaciones, "", valores)

In [81]:
for v in valores.keys():
  print(v + " " + str(valores[v]));

posibilidades 362880
max 77.0
min -69.0
valores enteros [-69.0, -68.0, -67.0, -66.0, -65.0, -64.0, -63.0, -62.0, -61.0, -60.0, -59.0, -58.0, -57.0, -56.0, -55.0, -54.0, -53.0, -52.0, -51.0, -50.0, -49.0, -48.0, -47.0, -46.0, -45.0, -44.0, -43.0, -42.0, -41.0, -40.0, -39.0, -38.0, -37.0, -36.0, -35.0, -34.0, -33.0, -32.0, -31.0, -30.0, -29.0, -28.0, -27.0, -26.0, -25.0, -24.0, -23.0, -22.0, -21.0, -20.0, -19.0, -18.0, -17.0, -16.0, -15.0, -14.0, -13.0, -12.0, -11.0, -10.0, -9.0, -8.0, -7.0, -6.0, -5.0, -4.0, -3.0, -2.0, -1.0, 0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0]
valores int

Calcula la complejidad del algoritmo por fuerza bruta

En este algoritmo vamos a considerar como operación unitaria la evaluación de una cadena, es decir, los nodos hoja que se han explorado en el árbol que forman las variaciones de estos elementos. Ya que se trata de un algoritmo iterativo y recursivo, podemos calcular la complejidad de la siguiente forma:

Siendo C el tamaño de la lista de cifras y siendo O el tamaño de la lista de operaciones, con C > O el número de operaciones realizadas sería:

- C , O = 0
- C * O * T((C-1), (O-1)) * ... * T((C-O), 0) , O > 0

Por tanto, podemos simplificar esta operación como:

- C * O * (C-1) * (O-1) * ... * C = (C! * O!) / (C-O-1)!

En el peor caso, donde C = O + 1 ya que siempre debe haber un digito almenos más que operaciones, la complejidad sería O(C!*(C-1)!). Es decir, tiene complejidad factorial, por lo que no es un algoritmo viable para su uso con valores altos.

(*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

Respuesta

(*)Calcula la complejidad del algoritmo 

Respuesta

Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

Respuesta

Aplica el algoritmo al juego de datos generado

Respuesta

Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

Respuesta

Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

Respuesta